# Web Scraping

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

verbose = True


## Elo Ratings

&#9632; For Fifa Ranking by year <br>
&#9632; Has data on Matches [Data can be grabbed from results.csv] <br> 
<ol>
    &#9633; Home/Away/Neutral <br>
    &#9633; Wins/Losses/Draws <br>
    </ol>

In [7]:
URL = 'https://eloratings.net/' 
requested = requests.get(URL) # get URL

if verbose: # print raw html content
    print('Raw HTML content \n', requested.content, '\n\n')

soup = BeautifulSoup(requested.content, 'lxml')

if verbose:
    print('Prettified Soup \n', soup.prettify(), '\n\n')

Raw HTML content 
 b'<!DOCTYPE html>\n<html lang="en"><head><title>World Football Elo Ratings</title>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n<meta name="description" content="Ratings for national football teams based on the Elo rating system.">\n<meta name="keywords" content="football, ratings, Elo, rankings, national, international, soccer, teams">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no">\n<link rel="stylesheet" href="scripts/slick.grid.css" type="text/css"/>\n<link rel="stylesheet" href="scripts/dygraph.css" type="text/css"/>\n<script type="text/javascript" src="scripts/dygraph.js"></script>\n<script type="text/javascript" src="scripts/jquery.js"></script>\n<script type="text/javascript" src="scripts/slick.core.js"></script>\n<script type="text/javascript" src="scripts/slick.grid.js"></script>\n<script type="text/javascript" src="scripts/cldr.js"></script>\n<script t


## Transfer Market

&#9632; Squad Size <br>
&#9632; Average Age <br> 
&#9632; Total Value <br>
&#9632; Points <br>
&#9632; Confederation <br>
<ol>
    &#9633; Valuable b/c could get confederation data to use <br>
    &#9633; Potential Issue Confederation stats only, CONACAF not as good as UEFA <br>
    </ol>

In [8]:
URL = 'https://www.transfermarkt.us/wettbewerbe/fifa' # URL of website w/ chart or data
requested = requests.get(URL) # get URL

if verbose: # print raw html content
    print('Raw HTML content \n', requested.content, '\n\n')

soup = BeautifulSoup(requested.content, 'lxml')

if verbose:
    print('Prettified Soup \n', soup.prettify(), '\n\n')

Raw HTML content 
 b'<html>\r\n<head><title>404 Not Found</title></head>\r\n<body bgcolor="white">\r\n<center><h1>404 Not Found</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n' 


Prettified Soup 
 <html>
 <head>
  <title>
   404 Not Found
  </title>
 </head>
 <body bgcolor="white">
  <center>
   <h1>
    404 Not Found
   </h1>
  </center>
  <hr/>
  <center>
   nginx
  </center>
 </body>
</html>
 





## Fifa Index

&#9632; Get Fifa video game team rankings <br>
<ol>
    &#9633; ATT <br>
    &#9633; MID <br>
    &#9633; DEF <br>
    &#9633; OVR <br>
    &#9633; Team Rating <br>
    </ol>
    
&#9632; How much of this is UNIQUE data? <br>
&#9632; Easier to calculate OVR? <br>

In [2]:
def get_Fifa_Index(URL, teamRank):
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.content, 'lxml')
    dom = etree.HTML(str(soup))
    # /a for team name and /span for int
    # tr[i]/td[j], where (i-2) is team rank, where j is column {2; Team Name, 4; ATT, 5; MID, 6; DEF, 7; OVR}  

    moreTeams = True
    currentRow = ['Name', 'ATT', 'MID', 'DEF']
    i=0
    while moreTeams:

        teamVal = i+3
        xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[' + str(teamVal) + ']/td[2]/a'
        try:
            #print('try:', i)
            currentRow[0]=(dom.xpath(xpath)[0].text)
            for j in range(4, 7):
                xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[' + str(teamVal) + ']/td[' + str(j) + ']/span'
                #print(xpath)
                currentRow[j-3]=(dom.xpath(xpath)[0].text)
            teamRank.append(currentRow)
            currentRow = ['Name', 'ATT', 'MID', 'DEF']
            count = 0
            i +=1
        except:
            #print('except: ', i)
            count += 1
            i += 1
            if count == 5:
                break
            pass

In [3]:
URL = ['https://www.fifaindex.com/teams/?page=1&type=1', 'https://www.fifaindex.com/teams/?page=2&type=1']
teamRank =[]
for i in range(len(URL)):
    get_Fifa_Index(URL[i], teamRank)
    
for x in range(len(teamRank)):
    print('%i, %s, %s, %s, %s' % (x, teamRank[x][0], teamRank[x][1], teamRank[x][2], teamRank[x][3]))

0, France, 85, 86, 83
1, Germany, 83, 85, 80
2, Portugal, 85, 83, 85
3, Spain, 83, 85, 83
4, England, 86, 83, 83
5, Argentina, 86, 81, 81
6, Italy, 84, 84, 82
7, Belgium, 83, 82, 80
8, Holland, 81, 82, 83
9, Brazil, 80, 80, 80
10, Denmark, 77, 78, 79
11, Poland, 81, 74, 75
12, Mexico, 80, 78, 74
13, Austria (National team), 79, 78, 78
14, Czech Republic, 79, 76, 76
15, Norway, 81, 76, 73
16, Sweden, 79, 76, 75
17, Ukraine, 76, 77, 74
18, Scotland, 73, 76, 77
19, United States, 75, 75, 74
20, Wales, 74, 74, 72
21, Greece, 73, 74, 73
22, Canada, 75, 73, 70
23, Hungary, 74, 69, 73
24, Republic of Ireland, 69, 73, 74
25, Romania, 72, 72, 68
26, Finland, 72, 72, 68
27, Australia, 75, 69, 70
28, China PR, 74, 70, 67
29, Iceland, 70, 68, 70
30, Northern Ireland, 65, 70, 71
31, New Zealand, 69, 67, 68



## Fifa Ranking

&#9632; Total Points <br>
&#9632; Ranking <br>

In [2]:
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import StaleElementReferenceException

def waitForLoad(driver):
    elem = driver.find_element_by_tag_name("html") 
    count = 0
    while True:
        count += 1
        if count > 20:
            print("Timing out after 10 seconds and returning")
            return
        time.sleep(.5) 
        try:
            elem == driver.find_element_by_tag_name("html") 
        except StaleElementReferenceException:
            return

In [10]:
URL = 'https://www.fifa.com/fifa-world-ranking/men?dateId=id13471'

#my_headers = {'accept-encoding':'gzip'}
headers = requests.head(URL)
print(headers)

<Response [401]>


In [6]:
URL = 'https://www.fifa.com/fifa-world-ranking/men?dateId=id13603'
webpage = requests.get(URL, headers={'Accept-Encoding': 'identity'})
soup = BeautifulSoup(webpage.text, 'lxml')
dom = etree.HTML(str(soup))
xpath = '//*[@id="content"]/main/section[2]/div/div/div[1]/table/tbody/tr[1]/td[3]/span[1]'
print(dom.xpath(xpath))


ContentDecodingError: ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check'))